In [15]:
from typing import List
import numpy as np
import pandas as pd
import pandas_market_calendars as mcal
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme(style="whitegrid")
from sklearn.metrics import mean_squared_error, median_absolute_error
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import GridSearchCV, PredefinedSplit
from sklearn.preprocessing import StandardScaler, QuantileTransformer
# from tensorflow.keras import Sequential, Model
# from tensorflow.keras.layers import Conv1D, MaxPool1D, Flatten, Dense, Dropout, Input
# from tensorflow.keras.layers import concatenate
# from tensorflow.keras.metrics import MeanSquaredError, RootMeanSquaredError
# from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
# from tensorflow.keras.utils import plot_model

#### define the tickers and indicators used

In [4]:
# exchange tickers
tickers = list([
    "SPY",  # S&P 500 Index Fund
    "IWV",  # Russell 3000 Index Fund
    "QQQ",  # Technology Sector Fund
    "IYF",  # Financials Sector Fund
    "XLP",  # Consumer Staples Sector Fund
    "XLU",  # Utilities Sector Funds
    "XLV",  # Health Care Sector Funds
    "IGE",  # NA Natural Resources ETF
    "XLE"  # Energy Sector Fund
])

# alias and FRED indicator
indicators = dict({
    "3M_TBILL": "DTB3",  # 3-Month Treasury Bill: Secondary Market Rate
    "CPI": "MEDCPIM158SFRBCLE",  # Median Consumer Price Index
    "VIX": "VIXCLS",  # CBOE Volatility Index
    "INDP": "INDPRO",  # Industrial Production: Total Index
    "USHY_ADJ": "BAMLH0A0HYM2",  # ICE BofA US High Yield Index Option-Adjusted Spread
    "US_LEADING": "USSLIND",  # Leading Index for the United States
    "30Y_FRMTG": "MORTGAGE30US",  # 30-Year Fixed Rate Mortgage Average in the United States
    "15Y_FRMTG": "MORTGAGE15US",  # 15-Year Fixed Rate Mortgage Average in the United States
    "CPI_URBAN": "CUSR0000SEHA",  # Consumer Price Index for All Urban Consumers: Rent of Primary Residence in U.S. City Average
    "RETAIL": "RSAFS",  # Advance Retail Sales: Retail and Food Services, Total
    "PHARMA": "PCU32543254",  # Producer Price Index by Industry: Pharmaceutical and Medicine Manufacturing
    "UNEMP": "UNRATE",  # Unemployment Rate
    "UNEMP_PERM": "LNS13026638",  # Unemployment Level - Permanent Job Losers
    "UNEMP_MEN": "LNS14000001",  # Unemployment Rate - Men
    "UNEMP_WMN": "LNS14000002",  # Unemployment Rate - Women
    "UNEMP_WHT": "LNS14000003",  # Unemployment Rate - White
    "UNEMP_BLK": "LNS14000006",  # Unemployment Rate - Black or African American
    "UNEMP_HIS": "LNS14000009",  # Unemployment Rate - Hispanic or Latino
    "INC": "PI",  # Personal Income
    "INC_DISP": "DSPIC96",  # Real Disposable Personal Income
    "INC_DISP_PC": "A229RX0",  # Real Disposable Personal Income: Per Capita
    "TAX_HIGH": "IITTRHB",  # U.S Individual Income Tax: Tax Rates for Regular Tax: Highest Bracket
    "TAX_LOW": "IITTRLB"  # U.S Individual Income Tax: Tax Rates for Regular Tax: Lowest Bracket
})

In [5]:
features = []
for ticker in tickers:
    for calculation in ['RET', 'VOL']:
        features.append(f'{ticker}_1D_{calculation}')
    for timeframe in ['1W', '1M', '3M', '6M']:
        for calculation in ['RET', 'STD', 'VOL', 'GBM']:
            features.append(f'{ticker}_{timeframe}_{calculation}')
    for calculation in ['RET', 'STD', 'VOL']:
        features.append(f'{ticker}_1Y_{calculation}')
for indicator in indicators.keys():
    features.append(indicator)

targets = [f'{ticker}_TARGET' for ticker in tickers]

#### Split the data into training, testing, and validation sets

In [7]:
# construct a dictionary with all market data in divided into sets and features/targets
dates = mcal.get_calendar('NYSE').schedule(start_date='2004-01-01', end_date='2020-12-31').index
market_data = dict({
    "X" : pd.read_pickle("data/market_data.zip").loc[:, features],
    "y" : pd.read_pickle("data/market_data.zip").loc[:, targets]
})

market_data["X_train"] = market_data["X"].loc['2004-01-01':'2015-12-31', :]
market_data["y_train"] = market_data["y"].loc['2004-01-01':'2015-12-31', :]
market_data["X_test"] = market_data["X"].loc['2016-01-01':'2020-12-31', :]
market_data["y_test"] = market_data["y"].loc['2016-01-01':'2020-12-31', :]
market_data["X"] = market_data["X"].loc['2004-01-01':'2020-12-31', :]
market_data["y"] = market_data["y"].loc['2004-01-01':'2020-12-31', :]

# Create split on train_all with -1 for training data and 0 for validation data (data after '2013-01-01')
split = PredefinedSplit(test_fold=[0 if v else -1 for v in market_data["X_train"].index < '2013-01-01'])

def fill_invalid(df, fill):
    df[df.isin([np.nan, np.inf, -1 * np.inf])] = fill

# Additing Quantile columns then normalizing and scaling
transformer = QuantileTransformer()
market_data["X_train"].loc[:, [column+"_QUANTILE" for column in market_data["X_train"].columns]] = \
    pd.DataFrame(transformer.fit_transform(market_data["X_train"]))
market_data["X_test"].loc[:, [column+"_QUANTILE" for column in market_data["X_test"].columns]] = \
    pd.DataFrame(transformer.transform(market_data["X_test"]))
market_data["X"].loc[:, [column+"_QUANTILE" for column in market_data["X"].columns]] = \
    pd.DataFrame(transformer.transform(market_data["X"]))

fill_invalid(market_data["X_train"], 0)
fill_invalid(market_data["X_test"], 0)
fill_invalid(market_data["X"], 0)

scaler = StandardScaler()
market_data["X_train"] = pd.DataFrame(scaler.fit_transform(market_data["X_train"]),
                                      columns=market_data["X_train"].columns, index=market_data["X_train"].index)
market_data["X_test"] = pd.DataFrame(scaler.transform(market_data["X_test"]),
                                      columns=market_data["X_test"].columns, index=market_data["X_test"].index)
market_data["X"] = pd.DataFrame(scaler.transform(market_data["X"]),
                                      columns=market_data["X"].columns, index=market_data["X"].index)

fill_invalid(market_data["X_train"], 0)
fill_invalid(market_data["X_test"], 0)
fill_invalid(market_data["X"], 0)

In [8]:
# Create a 3D input with each row being #time_steps x #feature_columns
def create_dataset_3D (X, y, time_steps = 21):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        Xs.append(np.array(X[i:i + time_steps, :]))
        ys.append(y[i + time_steps])
    return np.array(Xs), np.array(ys)

### Define CNN Models

In [ ]:
def build_CNN_simple():
    cnn = Sequential()
    cnn.add(Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(21, 424)))
    cnn.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
    cnn.add(Dropout(0.3))
    cnn.add(MaxPool1D(pool_size=2))
    cnn.add(Flatten())
    cnn.add(Dense(128, activation='relu'))
    cnn.add(Dropout(0.3))
    cnn.add(Dense(64, activation='relu'))
    cnn.add(Dropout(0.3))
    cnn.add(Dense(32, activation='relu'))
    cnn.add(Dropout(0.3))
    cnn.add(Dense(16, activation='relu'))
    cnn.add(Dropout(0.3))
    cnn.add(Dense(9))
    # Compile the model
    cnn.compile(optimizer='adam', loss='mean_squared_error',
                metrics=[MeanSquaredError(), RootMeanSquaredError()])
    return cnn


def build_CNN_multihead():
    input_1 = Input(shape=(21, 424))
    flat_1 = Flatten()(
        MaxPool1D(pool_size=2)(
            Dropout(0.3)(
                Conv1D(filters=32, kernel_size=3, activation='relu')(
                    Conv1D(filters=32, kernel_size=3, activation='relu')(input_1)))))
    input_2 = Input(shape=(21, 424))
    flat_2 = Flatten()(
        MaxPool1D(pool_size=2)(
            Dropout(0.3)(
                Conv1D(filters=32, kernel_size=7, activation='relu')(
                    Conv1D(filters=32, kernel_size=7, activation='relu')(input_2)))))
    input_3 = Input(shape=(21, 424))
    flat_3 = Flatten()(
        MaxPool1D(pool_size=2)(
            Dropout(0.3)(
                Conv1D(filters=32, kernel_size=11, activation='relu')(
                    Conv1D(filters=32, kernel_size=11, activation='relu')(input_3)))))
    input_4 = Input(shape=(21, 424))
    flat_4 = Flatten()(
        MaxPool1D(pool_size=2)(
            Dropout(0.3)(
                Conv1D(filters=16, kernel_size=3, activation='relu')(
                    Conv1D(filters=16, kernel_size=3, activation='relu')(input_4)))))
    input_5 = Input(shape=(21, 424))
    flat_5 = Flatten()(
        MaxPool1D(pool_size=2)(
            Dropout(0.3)(
                Conv1D(filters=64, kernel_size=3, activation='relu')(
                    Conv1D(filters=64, kernel_size=3, activation='relu')(input_5)))))
    merged_heads = concatenate([flat_1, flat_2, flat_3, flat_4, flat_5])
    dense_1 = Dense(128, activation='relu')(merged_heads)
    dropout_1 = Dropout(0.3) (dense_1)
    dense_2 = Dense(64, activation='relu')(dropout_1)
    dropout_2 = Dropout(0.3) (dense_2)
    dense_3 = Dense(32, activation='relu')(dropout_2)
    dropout_3 = Dropout(0.3) (dense_3)
    dense_4 = Dense(16, activation='relu')(dropout_3)
    dropout_4 = Dropout(0.3) (dense_4)
    output = Dense(9, activation='relu')(dropout_4)
    
    cnn = Model(inputs=[input_1, input_2, input_3, input_4, input_5], outputs=[output])
    # Compile the model
    cnn.compile(optimizer='adam', loss='mean_squared_error',
                metrics=[MeanSquaredError(), RootMeanSquaredError()])
    return cnn


plot_model(build_CNN_simple(), to_file='img/simpleCNN.jpeg', show_shapes=True, show_layer_names=True)
plot_model(build_CNN_multihead(), to_file='img/multi-headCNN.jpeg', show_shapes=True, show_layer_names=True)

### Fit and Test Models

In [ ]:
X_train, y_train = create_dataset_3D(np.array(market_data["X_train"]), np.array(market_data["y_train"]))
X_test, y_test = create_dataset_3D(np.array(market_data["X_test"]), np.array(market_data["y_test"]))

In [ ]:
simple_cnn = KerasRegressor(build_fn=build_CNN_simple, nb_epoch=1e8, batch_size=32, verbose=False)
simple_cnn.fit(X_train, y_train)

In [ ]:
y_train_pred_s = simple_cnn.predict(X_train)
y_test_pred_s = simple_cnn.predict(X_test)

In [ ]:
multihead_cnn = KerasRegressor(build_fn=build_CNN_multihead, nb_epoch=1e8, batch_size=32, verbose=False)
multihead_cnn.fit([X_train] * 5, y_train)  # 5 coppied inputs, 1 input for each head

In [ ]:
y_train_pred_mh = multihead_cnn.predict([X_train] * 5)
y_test_pred_mh = multihead_cnn.predict([X_test] * 5)

### Regression and Classification and Financial Reports

In [ ]:
def regression_metrics(y_true: np.array, y_pred: np.array, column_names: List[str]) -> np.array:
    mse_list = [mean_squared_error(y_true[:, column], y_pred[:, column])
                for column in range(len(column_names))]
    mae_list = [median_absolute_error(y_true[:, column], y_pred[:, column])
                for column in range(len(column_names))]
    print(f"Regression Metrics (Mean Squared Error and Median Absolute Error")
    for i, column in enumerate(column_names):
        print(f"- {column} Metrics\n"
              f"  - MSE: {mse_list[i]:.06f}\n"
              f"  - MAE: {mae_list[i]:.06f}\n"
              f"--------------------------------------------------")
    print(f"- Average\n"
          f"  - MSE: {float(np.mean(mse_list)):.06f}\n"
          f"  - MAE: {float(np.mean(mae_list)):.06f}\n"
          f"--------------------------------------------------")
    print(f"==================================================\n")


print("Training Set:")
regression_metrics(y_train, y_train_pred, tickers)
print("Testing Set:")
regression_metrics(y_test, y_test_pred, tickers)

In [ ]:
def classification_metrics(y_true: np.array, y_pred: np.array, column_names: List[str]) -> np.array:
    # create the positive negative classes in the targets
    _y_true = [[-1 if n < 0 else 1 for n in row] for row in y_true]
    _y_pred = [[-1 if n < 0 else 1 for n in row] for row in y_pred]

    n_col = len(column_names)
    cnf_mat_all = np.array([[0, 0], [0, 0]])
    mean_metrics = {"accuracy":0.0, "precision":0.0, "recall":0.0, "f1":0.0}
    confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
    print(f"Classification Metrics (Mean Squared Error and Median Absolute Error")
    for i, column in enumerate(column_names):
        cnf_mat = confusion_matrix(_y_true[:, column], _y_pred[:, column] )
        accuracy = accuracy_score(_y_true[:, column], _y_pred[:, column] )
        precision = precision_score(_y_true[:, column], _y_pred[:, column] )
        recall = recall_score(_y_true[:, column], _y_pred[:, column] )
        f1 = f1_score(_y_true[:, column], _y_pred[:, column] )
        print(f"- {column} Metrics\n"
              f"  - Confusion Matrix:\n"
              f"    [{cnf_mat[0]},\n"
              f"     {cnf_mat[1]}]\n"
              f"  - Accuracy:  {accuracy:.06f}\n"
              f"  - Precision: {precision:.06f}\n"
              f"  - Recall:    {recall:.06f}\n"
              f"  - F1-Score:  {f1:.06f}\n"
              f"--------------------------------------------------")
        cnf_mat_all = cnf_mat_all + np.array(cnf_mat)
        mean_metrics["accuracy"] += (accuracy / n_col)
        mean_metrics["precision"] += (precision / n_col)
        mean_metrics["recall"] += (recall / n_col)
        mean_metrics["f1"] += (f1 / n_col)

    print(f"- Average\n"
          f"  - Confusion Matrix:\n"
          f"    [{cnf_mat_all[0]},\n"
          f"     {cnf_mat_all[1]}]\n"
          f"  - Accuracy:  {mean_metrics["accuracy"]:.06f}\n"
          f"  - Precision: {mean_metrics["precision"]:.06f}\n"
          f"  - Recall:    {mean_metrics["recall"]:.06f}\n"
          f"  - F1-Score:  {mean_metrics["f1"]:.06f}\n"
          f"--------------------------------------------------")
    print(f"==================================================\n")


print("Training Set:")
classification_metrics(y_train, y_train_pred, tickers)
print("Testing Set:")
classification_metrics(y_test, y_test_pred, tickers)

In [11]:
def cumulative_return(returns_df: pd.DataFrame) -> pd.DataFrame:
    """
    Calculate the cumulative returns for all columns in the returns DataFrame
    :param returns_df: the DataFrame of returns
    :return: a DataFrame of cumulative returns for each column
    """
    cumulative_return_df = returns_df.fillna(0) + 1
    return cumulative_return_df.cumprod(axis=0)


def annualized_return(returns_df: pd.DataFrame) -> pd.DataFrame:
    """
    Calculate an return the annualized return for all daily return columns in the given DataFrame
    :param returns_df: the DataFrame of returns
    :return: a series of annualized returns for each column
    """
    n_days = len(returns_df.index)
    returns = returns_df.add(1).prod(axis=0)
    # annualized with the number of days and the total days per trading year
    returns = (returns ** (252/n_days)).add(-1)
    return returns


def financial_metrics(y_true: np.array, y_pred: np.array, column_names: List[str], time_index: pd.DatetimeIndex,
                      allocation: float=0.05) -> np.array:
    """
    Takes a list of true returns and predicted returns. Uses the predicted returns to generate 5% allocation signals
    to long/short the given asset. 
    Plots the adjusted returns of the columns individually, then the portfolio as a whole 
    (row mean of returns 2D array).
    """
    signals = np.array([[-allocation if n < 0 else allocation for n in row] for row in y_pred])
    adj_returns = y_true * signals
    
    # create dataframes of returns from the parameters and column names / dates
    # divide returns by 21 (because each day is a 1-month look-ahead return)
    df_b = pd.DataFrame(y_true / 21, columns=column_names, index=time_index)
    df_b["Portfolio"] = df_b.mean(axis=1)
    df_m = pd.DataFrame(adj_returns / 21, columns=column_names, index=time_index)
    df_m["Portfolio"] = df_m.mean(axis=1)
    
    # generate annualized returns
    df_b_ar = annualized_return(df_b)
    df_m_ar = annualized_return(df_m)

    # generate cumulative returns
    df_b_c = cumulative_return(df_b)
    df_m_c = cumulative_return(df_m)
    
    for column in df_b.columns:
        df_plot = df_b.loc[:,[column]]
        df_plot[column + " Model"] = df_m.loc[column]
        f, ax = plt.subplots(figsize=(11, 9))
        lineplot = sns.lineplot(data=df_plot, x="Time", y="Cumulative Return (Starting = 1)")
        lineplot.set_title(f"{column} Cumulative Return")
        fig = lineplot.get_figure()
        fig.savefig(f"img/{column}-return-cnn.jpeg")
        print(f"- {column} Metrics\n"
              f"  - Annualized Return: \n"
              f"    - Benchmark: {df_b_ar[column]}\n"
              f"    - Model:     {df_m_ar[column]}\n"
              f"  - Volatility: \n"
              f"    - Benchmark: {float(np.std(df_b[column])):.06f}\n"
              f"    - Model:     {float(np.std(df_m[column])):.06f}\n"
              f"--------------------------------------------------")
        
    print(f"==================================================\n")



[[0.05, 0.05, -0.05, -0.05, 0.05, -0.05], [0.05, -0.05, 0.05, -0.05, -0.05, 0.05]]


### Metrics over Number of Epochs

In [ ]:
powers = [i for i in range(1, 10)]
epochs = sorted([5* 10**i for i in powers] + [10**i for i in powers])
epoch_metrics = []
for epoch in epochs:
    epoch_model = KerasRegressor(build_fn=build_CNN, nb_epoch=epoch, batch_size=32, verbose=False)
    epoch_model.fit(X_train, y_train)
    y_train_pred = epoch_model.predict(X_train)
    y_test_pred = epoch_model.predict(X_test)
    mse_train_avg = np.mean([mean_squared_error(y_train[:, column], y_train_pred[:, column])
                for column in range(len(tickers))])
    mae_train_avg = np.mean([median_absolute_error(y_train[:, column], y_train_pred[:, column])
                for column in range(len(tickers))])
    
    y_test_pred = epoch_model.predict(X_test)
    y_test_pred = epoch_model.predict(X_test)
    mse_test_avg = np.mean([mean_squared_error(y_test[:, column], y_test_pred[:, column])
                for column in range(len(tickers))])
    mae_test_avg = np.mean([median_absolute_error(y_test[:, column], y_test_pred[:, column])
                for column in range(len(tickers))])
    epoch_metrics.append({"Epochs": epoch, "MSE": mse_train_avg, "MAE": mae_train_avg, "Set": "Train"})
    epoch_metrics.append({"Epochs": epoch, "MSE": mse_test_avg, "MAE": mae_test_avg, "Set": "Test"})
epoch_metrics = pd.DataFrame(epoch_metrics)
epoch_metrics.set_index("Epochs", inplace=True)

In [ ]:
plt.subplot(2, 1, 1)
plt.semilogx(epochs, epoch_metrics.where(epoch_metrics.Set == "Train").dropna().MSE, label='Train')
plt.semilogx(epochs, epoch_metrics.where(epoch_metrics.Set == "Test").dropna().MSE, label='Test')

plt.legend(loc='upper left')
plt.ylim([0.0025, 0.01])
plt.xlabel('Number of Epochs')
plt.ylabel('Average MSE')

# Show estimated coef_ vs true coef

plt.subplot(2, 1, 2)
plt.semilogx(epochs, epoch_metrics.where(epoch_metrics.Set == "Train").dropna().MAE, label='Train')
plt.semilogx(epochs, epoch_metrics.where(epoch_metrics.Set == "Test").dropna().MAE, label='Test')

plt.legend(loc='upper left')
plt.ylim([0.025, 0.05])
plt.xlabel('Number of Epochs')
plt.ylabel('Average MAE')
plt.show()

### Metrics over Batch Size

In [ ]:
batches = [8, 16, 32, 64, 128, 256, 512]
batch_metrics = []
for batch in batches:
    batch_model = KerasRegressor(build_fn=build_CNN, nb_epoch=1e8, batch_size=batch, verbose=False)
    batch_model.fit(X_train, y_train)
    y_train_pred = batch_model.predict(X_train)
    y_test_pred = batch_model.predict(X_test)
    mse_train_avg = np.mean([mean_squared_error(y_train[:, column], y_train_pred[:, column])
                for column in range(len(tickers))])
    mae_train_avg = np.mean([median_absolute_error(y_train[:, column], y_train_pred[:, column])
                for column in range(len(tickers))])
    
    y_test_pred = batch_model.predict(X_test)
    y_test_pred = batch_model.predict(X_test)
    mse_test_avg = np.mean([mean_squared_error(y_test[:, column], y_test_pred[:, column])
                for column in range(len(tickers))])
    mae_test_avg = np.mean([median_absolute_error(y_test[:, column], y_test_pred[:, column])
                for column in range(len(tickers))])
    batch_metrics.append({"Batch Size": batch, "MSE": mse_train_avg, "MAE": mae_train_avg, "Set": "Train"})
    batch_metrics.append({"Batch Size": batch, "MSE": mse_test_avg, "MAE": mae_test_avg, "Set": "Test"})
batch_metrics = pd.DataFrame(batch_metrics)
batch_metrics.set_index("Batch Size", inplace=True)

In [ ]:
plt.subplot(2, 1, 1)
plt.semilogx(batches, batch_metrics.where(batch_metrics.Set == "Train").dropna().MSE, label='Train')
plt.semilogx(batches, batch_metrics.where(batch_metrics.Set == "Test").dropna().MSE, label='Test')

plt.legend(loc='upper left')
plt.ylim([0.0025, 0.01])
plt.xlabel('Batch Size')
plt.ylabel('Average MSE')

# Show estimated coef_ vs true coef

plt.subplot(2, 1, 2)
plt.semilogx(batches, batch_metrics.where(batch_metrics.Set == "Train").dropna().MAE, label='Train')
plt.semilogx(batches, batch_metrics.where(batch_metrics.Set == "Test").dropna().MAE, label='Test')

plt.legend(loc='upper left')
plt.ylim([0.025, 0.05])
plt.xlabel('Batch Size')
plt.ylabel('Average MAE')
plt.show()